In [ ]:
import numpy as np
import tensorflow as tf
from scipy import misc
import glob
import imageio
import matplotlib.pyplot as plt
import random
import csv
 
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.00001
batch_size = 32
display_step = batch_size*10
hm_epochs=30
BS=batch_size
picsize=256
inputdepth=7
stvs=0.01
trainmode=True
localpath='/path/to/dir/Knee_MRI/Femur/Train/'

num_input = picsize*picsize 
num_classes = 2 

# tf Graph input
with tf.name_scope("Input"):
    X = tf.placeholder(tf.float32, [None, picsize,picsize,inputdepth])
with tf.name_scope("True_Class"):
    Y = tf.placeholder(tf.float32, [None, num_classes])
with tf.name_scope("Blanks"):
    num_classes_tf=tf.constant(num_classes,tf.int32)
with tf.name_scope("true_labels"):
    true_label=tf.placeholder(tf.float32, [1])

    
# Simonyan K, Zisserman A. Very deep convolutional networks for large-scale image recognition. 
# arXiv preprint arXiv:1409.1556. 2014 Sep 4.    

In [ ]:
#Image store
runmode='jupyter'

if runmode=='jupyter':
    imdpath=localpath + "ImdTrain/"
    imdfiles=glob.glob(localpath + "ImdTrain/*.png")
    numfiles=int(len(glob.glob(localpath "ImdTrain/*.png"))/inputdepth)
    root=localpath
    
    imdpath=localpath + "ImdVal/"
    imdfiles=glob.glob(localpath + "ImdVal/*.png")
    numfiles=int(len(glob.glob(localpath "ImdVal/*.png"))/inputdepth)
    root=localpath
    
else: 
    imdpath="/mydata/Femur/Train/ImdTrain/"
    imdfiles=glob.glob("/mydata/Femur/Train/ImdTrain/*.png")
    numfiles=int(len(glob.glob("/mydata/Femur/Train/ImdTrain/*.png"))/inputdepth)
    root='/mydata/'
    
    imdpath_val="/mydata/Femur/Val/ImdVal/"
    imdfiles_val=glob.glob("/mydata/Femur/Val/ImdVal/*.png")
    numfiles_val=int(len(glob.glob("/mydata/Femur/Val/ImdVal/*.png"))/inputdepth)
    root_val='/mydata/'
    

print(numfiles)

x = ([[i] for i in range(numfiles)])
shuflist=random.sample(x,len(x))

#print(imdfiles[0])
#print(pxdfiles[0])
tempfiles_new=[]

s=''
for i in range(numfiles):
    if len(str(shuflist[i][0]+1))==1:
        templist=('0000', str(shuflist[i][0]+1)) 
        tempfiles_new.append(s.join(templist))
    elif len(str(shuflist[i][0]+1))==2:
        templist=('000', str(shuflist[i][0]+1)) 
        tempfiles_new.append(s.join(templist)) 
    elif len(str(shuflist[i][0]+1))==3:
        templist=('00', str(shuflist[i][0]+1)) 
        tempfiles_new.append(s.join(templist))  
    elif len(str(shuflist[i][0]+1))==4:
        templist=('0', str(shuflist[i][0]+1)) 
        tempfiles_new.append(s.join(templist)) 
    else:
        templist=(str(shuflist[i][0]+1)) 
        tempfiles_new.append(s.join(templist))
        
print(tempfiles_new[:10])
print((imdpath + str(tempfiles_new[0]) + 'A.png'))

In [ ]:
#Class data for classification of slices is stored as either 0 or 1 (not a femur, or femur) in a .csv 
#where each row corresponds to slice's file identifier in the data store

classtrue=[]
path=(root + "GroundList.csv")
print(path)
f = open(path, 'r')
reader = csv.reader(f)
for row in reader:
    classtrue.append(row)
f.close()

print(len(classtrue))
print(classtrue[:10])
print(int(classtrue[3][0]))


In [ ]:
classtrue_val=[]
path=(root + "Femur/Val/fem_mri_classlist_Val.csv")
print(path)
f = open(path, 'r')
reader = csv.reader(f)
for row in reader:
    classtrue_val.append(row)
f.close()

print(len(classtrue_val))
print(classtrue_val[:10])
print(int(classtrue_val[3][0]))

In [ ]:
def conv_layer(inputs, channels_in, channels_out,is_training,stvs,strides=1,scopename="conv"):
    with tf.name_scope(scopename):
        w=tf.Variable(tf.random_normal([3, 3, channels_in, channels_out],stddev=stvs))
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, w)
            
        b=tf.Variable(tf.random_normal([channels_out],stddev=stvs))
        tf.summary.histogram(("Weight" + scopename),w)
        tf.summary.histogram(("Bias" + scopename),b)
        x = tf.nn.conv2d(inputs, w, strides=[1, strides, strides, 1], padding='SAME')
        x = tf.nn.bias_add(x, b)
        x=batch_norm_wrapper(x, is_training, decay = 0.99)
        return tf.nn.relu(x)

def maxpool2d(x, k=2,scopename="pool"):
    with tf.name_scope(scopename):
        # MaxPool2D wrapper
        return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='VALID')
    
# Batch normalization wrapper to distinguish between training and testing
# SOURCE: https://r2rt.com/implementing-batch-normalization-in-tensorflow.html
def batch_norm_wrapper(inputs, is_training, decay = 0.99):
    
    epsilon = 1e-3
    scale = tf.Variable(tf.ones([inputs.get_shape()[-1]]))
    beta = tf.Variable(tf.zeros([inputs.get_shape()[-1]]))
    pop_mean = tf.Variable(tf.zeros([inputs.get_shape()[-1]]), trainable=False)
    pop_var = tf.Variable(tf.ones([inputs.get_shape()[-1]]), trainable=False)

    if is_training:
        batch_mean, batch_var = tf.nn.moments(inputs,[0,1,2])
        train_mean = tf.assign(pop_mean,
                               pop_mean * decay + batch_mean * (1 - decay))
        train_var = tf.assign(pop_var,
                              pop_var * decay + batch_var * (1 - decay))
        with tf.control_dependencies([train_mean, train_var]):
            return tf.nn.batch_normalization(inputs,
                batch_mean, batch_var, beta, scale, epsilon)
    else:
        return tf.nn.batch_normalization(inputs,
            pop_mean, pop_var, beta, scale, epsilon)

In [ ]:
#Define Architecture
def conv_net(x, inputdepth, is_training):
    stvs=0.01
    depthstart=64
    num_classes=2
    if is_training==True:
        keepprob=0.5
    else:
        keepprob=1
    
    #Fit batches into correct shape
    x = tf.reshape(x, shape=[-1, picsize, picsize, inputdepth]) 
    
    conv1a = conv_layer(x, inputdepth, depthstart, is_training, stvs,scopename="conv1a")
    conv1b = conv_layer(conv1a, depthstart, depthstart, is_training, stvs, scopename="conv1b")
    pooled1 = maxpool2d(conv1b,scopename="pooled1")
        
    conv2a = conv_layer(pooled1, depthstart, depthstart*2, is_training, stvs,scopename="conv2a")
    conv2b = conv_layer(conv2a, depthstart*2, depthstart*2, is_training, stvs, scopename="conv2b")
    pooled2 = maxpool2d(conv2b,scopename="pooled2")
    
    conv3a = conv_layer(pooled2, depthstart*2, depthstart*4, is_training, stvs,scopename="conv3a")
    conv3b = conv_layer(conv3a, depthstart*4, depthstart*4, is_training, stvs, scopename="conv3b")
    conv3c = conv_layer(conv3b, depthstart*4, depthstart*4, is_training, stvs, scopename="conv3c")
    pooled3 = maxpool2d(conv3c,scopename="pooled3")
    
    conv4a = conv_layer(pooled3, depthstart*4, depthstart*8, is_training, stvs,scopename="conv4a")
    conv4b = conv_layer(conv4a, depthstart*8, depthstart*8, is_training, stvs, scopename="conv4b")
    conv4c = conv_layer(conv4b, depthstart*8, depthstart*8, is_training, stvs, scopename="conv4c")
    pooled4 = maxpool2d(conv4c,scopename="pooled4")
    
    #Dense layers
    with tf.name_scope("Dense1"):
        dense1nodes=1000
        flatten=tf.reshape(pooled4,[-1,(depthstart*8)*16*16])
        dense1vars={'weights':tf.Variable(tf.random_normal([(depthstart*8)*16*16, dense1nodes],stddev=stvs)), 
                'biases':tf.Variable(tf.random_normal([dense1nodes],stddev=stvs))}
        
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, dense1vars['weights'])
        
        dense1=tf.add(tf.matmul(flatten,dense1vars['weights']),dense1vars['biases'])
        dense1=tf.nn.relu(dense1)
        dropout1=tf.nn.dropout(dense1,keepprob)
        
    with tf.name_scope("Dense2"):
        dense2nodes=1000
        dense2vars={'weights':tf.Variable(tf.random_normal([dense1nodes, dense2nodes],stddev=stvs)), 
                'biases':tf.Variable(tf.random_normal([dense2nodes],stddev=stvs))}
        
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, dense2vars['weights'])
        dense2=tf.add(tf.matmul(dropout1,dense2vars['weights']),dense2vars['biases'])
        dense2=tf.nn.relu(dense2)
        dropout2=tf.nn.dropout(dense2,keepprob)
    
    #Softmax for prediction
    with tf.name_scope("Out_Layer"):
        outvars={'weights':tf.Variable(tf.random_normal([dense2nodes, num_classes],stddev=stvs)), 
                'biases':tf.Variable(tf.random_normal([num_classes],stddev=stvs))}
        
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, outvars['weights'])
        outlayer=tf.add(tf.matmul(dropout2,outvars['weights']),outvars['biases'])
        print(outlayer)
        
    return outlayer

In [ ]:
#Retrieve true class
def Get_Ground(in_val,num_classes_tf_in):
    if not in_val.dtype == tf.float32:
        in_val = tf.cast( in_val, tf.float32)
        
    if not num_classes_tf_in.dtype == tf.int32:
        num_classes_tf_in = tf.cast(num_classes_tf_in, tf.int32)
        
    max_val=tf.reduce_max(in_val)
    
    if max_val==0:
        return tf.one_hot(in_val,num_classes_tf_in)
    else:
        norm_val=tf.divide(in_val,max_val)
        return tf.one_hot(tf.cast(norm_val,tf.uint8),num_classes_tf_in)

In [ ]:
#Define flow graph
logits = conv_net(X, inputdepth, trainmode)

with tf.name_scope("Get_Label"):
    label_1H=Get_Ground(true_label,num_classes_tf)

#Prediction function for evaluating accuracy
with tf.name_scope("Softmax"):
    prediction = tf.nn.softmax(logits)
    
#Define Loss
with tf.name_scope("Loss"):
    loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
    
    #regularizer = tf.contrib.layers.l2_regularizer(scale=0.0001)
    #reg_variables = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #reg_term = tf.contrib.layers.apply_regularization(regularizer, reg_variables)
    #loss_op += reg_term
    
    tf.summary.scalar("Loss",loss_op)

#Define optimizer
with tf.name_scope("Optimizer"):
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)   
    
    train_op = optimizer.minimize(loss_op)

# Evaluate model
with tf.name_scope("Accuracy"):
    correct_pred = tf.equal(tf.argmax(prediction, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    tf.summary.scalar("accuracy",accuracy)

#Confusion matrix
with tf.name_scope("Confusion_Matrix"):
    batch_confusion = tf.confusion_matrix(tf.reshape(tf.argmax(Y,axis=1),[-1]),tf.reshape(tf.argmax(prediction,axis=1),[-1]),num_classes=num_classes,name='batch_confusion')
    confusion_image = tf.reshape( tf.cast(batch_confusion, tf.float32),[1, num_classes, num_classes, 1])
    tf.summary.image('confusion',confusion_image)

#Define writer for Tensorboard
writer=tf.summary.FileWriter("/output/3")
summ=tf.summary.merge_all()

# Initialize the variablesS
init = tf.global_variables_initializer()

#Define saver for model saver
saver = tf.train.Saver()

In [ ]:
#Send it
ct=0
with tf.Session() as sess:
    sess.run(init)
    writer.add_graph(sess.graph)

    #saver.restore(sess, '/models/model_30112') #Option to restore a model to resume training
    
    for epoch in range(hm_epochs):
        
        x = ([[i] for i in range(numfiles)])
        shuflist=random.sample(x,len(x))
        tempfiles_new=[]
        s=''
        for i in range(numfiles):
            if len(str(shuflist[i][0]+1))==1:
                templist=('0000', str(shuflist[i][0]+1)) 
                tempfiles_new.append(s.join(templist))
            elif len(str(shuflist[i][0]+1))==2:
                templist=('000', str(shuflist[i][0]+1)) 
                tempfiles_new.append(s.join(templist)) 
            elif len(str(shuflist[i][0]+1))==3:
                templist=('00', str(shuflist[i][0]+1)) 
                tempfiles_new.append(s.join(templist))  
            elif len(str(shuflist[i][0]+1))==4:
                templist=('0', str(shuflist[i][0]+1)) 
                tempfiles_new.append(s.join(templist)) 
            else:
                templist=(str(shuflist[i][0]+1)) 
                tempfiles_new.append(s.join(templist))


        i=0
        print("EPOCH " + "{:d}".format(epoch))
        while i+BS<numfiles:    
            start=i; end=i+BS;
            bx=np.empty((BS,picsize,picsize,inputdepth))
            by=np.empty((BS,num_classes))
            ground=np.empty((BS,picsize,picsize))
            for j in range(BS): # Import data for that mini-batch
                bx[j,:,:,0]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'A.png')) #Import CT slice
                bx[j,:,:,1]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'B.png')) #Import CT slice
                bx[j,:,:,2]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'C.png')) #Import CT slice
                bx[j,:,:,3]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'D.png')) #Import CT slice
                bx[j,:,:,4]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'E.png')) #Import CT slice
                bx[j,:,:,5]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'F.png')) #Import CT slice
                bx[j,:,:,6]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'G.png')) #Import CT slice
                
                bx[j,:,:,:]=bx[j,:,:,:]-np.mean(bx[j,:,:,:],axis=(0,1,2))

                by[j,:]=sess.run(label_1H,feed_dict={true_label: np.array(classtrue[int(tempfiles_new[i+j])-1][0]).reshape(1)}) #Create one hot tensor   
                

            #After setting up batch_x and batch_y, we can run train_op
            sess.run(train_op,feed_dict={X: bx, Y: by})
            loss, acc, summary=sess.run([loss_op,accuracy,summ],feed_dict={X: bx, Y: by})
            
            
            #conf_im=sess.run(confusion_image,feed_dict={X: bx, Y: by})
            writer.add_summary(summary, ct)
            
            
            if i%display_step==0 or i==BS or i==0: 
                s=''
                
                checkpointnamelist=('./model_',str(ct))
                checkpointname= s.join(checkpointnamelist)
                save_path = saver.save(sess,checkpointname)
                
                #print("Model saved in file: %s" % save_path)
                print(" Step " + str(i) + ", Minibatch Loss= " + "{:.4f}".format(loss) + ", Training Accuracy = " + "{:.3f}".format(acc)) 
                print(sess.run(batch_confusion,feed_dict={X: bx, Y: by}))
                
            if i%(display_step*5)==0 and i>0:
                #Validation step
                acc_vec_val=[]; ct_val=0;
                for j in range(numfiles_val): #Import data for that mini-batch
                    #for j in range(100): #Import data for that mini-batch

                    #Validation step -- Load entire validation set
                    bx=np.empty((1,picsize,picsize,inputdepth))
                    by=np.empty((1,num_classes))
                    batch_ID=0;
                    #Import input data
                    ID=str(j)
                    while(len(ID))<5:
                        ID='0'+ID

                    bx[j,:,:,0]=imageio.imread((imdpath_val + ID + 'A.png')) #Import CT slice
                    bx[j,:,:,1]=imageio.imread((imdpath_val + ID + 'B.png')) #Import CT slice
                    bx[j,:,:,2]=imageio.imread((imdpath_val + ID + 'C.png')) #Import CT slice
                    bx[j,:,:,3]=imageio.imread((imdpath_val + ID + 'D.png')) #Import CT slice
                    bx[j,:,:,4]=imageio.imread((imdpath_val + ID + 'E.png')) #Import CT slice
                    bx[j,:,:,5]=imageio.imread((imdpath_val + ID + 'F.png')) #Import CT slice
                    bx[j,:,:,6]=imageio.imread((imdpath_val + ID + 'G.png')) #Import CT slice        
                    bx[batch_ID,:,:,:]=bx[batch_ID,:,:,:]-np.mean(bx[batch_ID,:,:,:],axis=(0,1,2))

                    by[batch_ID,:]=ground_val[int(ID)]  
                    #After setting up batch_x and batch_y, we can run train_op
                    loss, acc, summary=sess.run([loss_op,accuracy,summ],feed_dict={X: bx, Y: by})
                    BC=sess.run(batch_confusion,feed_dict={X: bx, Y: by})
                    acc_vec_val.append(acc)
                    
                print("Validation Accuracy: " + "{:.4f}".format(np.mean(acc_vec_val)))

                
            i=i+BS
            ct=ct+BS
        
    print("We out here")

In [ ]:
testmode='yes'
if testmode=='yes':
    imdpath="/mydata/ImdSag/"
    imdfiles=glob.glob("/mydata/ImdSag/*.png")
    numfiles=int(len(glob.glob("/mydata/ImdSag/*.png"))/inputdepth)
    root='/mydata/'
    print(numfiles)
    x = ([[i] for i in range(numfiles)])
    shuflist=random.sample(x,len(x))
    tempfiles_new=[]
    s=''
    for i in range(numfiles):
        if len(str(shuflist[i][0]+1))==1:
            templist=('0000', str(shuflist[i][0]+1)) 
            tempfiles_new.append(s.join(templist))
        elif len(str(shuflist[i][0]+1))==2:
            templist=('000', str(shuflist[i][0]+1)) 
            tempfiles_new.append(s.join(templist)) 
        elif len(str(shuflist[i][0]+1))==3:
            templist=('00', str(shuflist[i][0]+1)) 
            tempfiles_new.append(s.join(templist))  
        elif len(str(shuflist[i][0]+1))==4:
            templist=('0', str(shuflist[i][0]+1)) 
            tempfiles_new.append(s.join(templist)) 
        else:
            templist=(str(shuflist[i][0]+1)) 
            tempfiles_new.append(s.join(templist))

    print(tempfiles_new[:10])
    print((imdpath + str(tempfiles_new[0]) + 'A.png'))
        
        
    #Sess time
    TP=0; TN=0; FP=0; FN=0;
    ct=0
    BS=32
    with tf.Session() as sess:
        sess.run(init)
        saver.restore(sess, '/models/model_87232')
        writer.add_graph(sess.graph)

        i=0
        print("Testing")
        while i+BS<numfiles+1:    
            start=i; end=i+BS;
            bx=np.empty((BS,picsize,picsize,inputdepth))
            by=np.empty((BS,num_classes))
            ground=np.empty((BS,picsize,picsize)) 
            for j in range(BS): # Import data for that mini-batch
                bx[j,:,:,0]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'A.png')) #Import CT slice
                bx[j,:,:,1]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'B.png')) #Import CT slice
                bx[j,:,:,2]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'C.png')) #Import CT slice
                bx[j,:,:,3]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'D.png')) #Import CT slice
                bx[j,:,:,4]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'E.png')) #Import CT slice
                bx[j,:,:,5]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'F.png')) #Import CT slice
                bx[j,:,:,6]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'G.png')) #Import CT slice

                by[j,:]=sess.run(label_1H,feed_dict={true_label: np.array(classtrue[int(tempfiles_new[i+j])-1][0]).reshape(1)}) #Create one hot tensor   

            acc, conf=sess.run([accuracy,batch_confusion],feed_dict={X: bx, Y: by})
            print(acc)
            print(conf)
            TP+=conf[1,1]
            TN+=conf[0,0]
            
            FP+=conf[1,0]
            FN+=conf[0,1]
            i=i+BS
        
        BS=26
        while i+BS<numfiles+1:    
            start=i; end=i+BS;
            bx=np.empty((BS,picsize,picsize,inputdepth))
            by=np.empty((BS,num_classes))
            ground=np.empty((BS,picsize,picsize)) 
            for j in range(BS): # Import data for that mini-batch
                bx[j,:,:,0]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'A.png')) #Import CT slice
                bx[j,:,:,1]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'B.png')) #Import CT slice
                bx[j,:,:,2]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'C.png')) #Import CT slice
                bx[j,:,:,3]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'D.png')) #Import CT slice
                bx[j,:,:,4]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'E.png')) #Import CT slice
                bx[j,:,:,5]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'F.png')) #Import CT slice
                bx[j,:,:,6]=imageio.imread((imdpath + str(tempfiles_new[i+j]) + 'G.png')) #Import CT slice

                by[j,:]=sess.run(label_1H,feed_dict={true_label: np.array(classtrue[int(tempfiles_new[i+j])-1][0]).reshape(1)}) #Create one hot tensor   

            acc, conf=sess.run([accuracy,batch_confusion],feed_dict={X: bx, Y: by})
            TP+=conf[1,1]
            TN+=conf[0,0]
            FP+=conf[1,0]
            FN+=conf[0,1]
            i=i+BS
            
    print((TP+TN)/(TP+TN+FP+FN))
    
            
    